# Fine-Tuning: TechGadgets Support Bot

This notebook handles the fine-tuning process:
1. Upload training and validation files to OpenAI
2. Create a fine-tuning job with specified parameters
3. Monitor the training progress
4. Retrieve the fine-tuned model ID

## Step 1: Setup and Configuration

In [6]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Fine-tuning configuration
FINE_TUNING_CONFIG = {
    "model": "gpt-4o-mini-2024-07-18",
    "training_file": None,  # Will be set after upload
    "validation_file": None,  # Will be set after upload
    "hyperparameters": {
        "n_epochs": 1,
        "batch_size": 1,
        "learning_rate_multiplier": None  # Auto
    },
    "suffix": "techgadgets-support",
    "seed": 42
}

print("✅ OpenAI client initialized")
print(f"Model: {FINE_TUNING_CONFIG['model']}")
print(f"Suffix: {FINE_TUNING_CONFIG['suffix']}")

✅ OpenAI client initialized
Model: gpt-4o-mini-2024-07-18
Suffix: techgadgets-support


## Step 2: Upload Training and Validation Files

### 💡 Fine-tuning requires a paid OpenAI account

Fine-tuning uses billable tokens. If you see **"You exceeded your current quota"**:

1. **Check billing**: [OpenAI Billing](https://platform.openai.com/account/billing) — add a payment method or top up credits.
2. **Check usage**: [OpenAI Usage](https://platform.openai.com/usage) — confirm your plan allows fine-tuning.
3. **Pricing**: [Fine-tuning pricing](https://openai.com/api/pricing/) — e.g. gpt-4o-mini fine-tuning is charged per token.

Free tier / trial credits usually do **not** include fine-tuning; you need a paid plan with available balance.

In [7]:
# Upload training file
print("Uploading training file...")
with open("../data/training_data.jsonl", "rb") as f:
    training_file = client.files.create(
        file=f,
        purpose="fine-tune"
    )
    FINE_TUNING_CONFIG["training_file"] = training_file.id
    print(f"✅ Training file uploaded: {training_file.id}")
    print(f"   File name: {training_file.filename}")
    print(f"   File size: {training_file.bytes} bytes")

# Wait a moment for file processing
time.sleep(2)

# Upload validation file
print("\nUploading validation file...")
with open("../data/validation_data.jsonl", "rb") as f:
    validation_file = client.files.create(
        file=f,
        purpose="fine-tune"
    )
    FINE_TUNING_CONFIG["validation_file"] = validation_file.id
    print(f"✅ Validation file uploaded: {validation_file.id}")
    print(f"   File name: {validation_file.filename}")
    print(f"   File size: {validation_file.bytes} bytes")

Uploading training file...
✅ Training file uploaded: file-HubMu4aeHoD528JmrpVo4S
   File name: training_data.jsonl
   File size: 483064 bytes

Uploading validation file...
✅ Validation file uploaded: file-G7JGvB35BHgFqeuYeCZsWR
   File name: validation_data.jsonl
   File size: 119054 bytes


## Step 3: Create Fine-Tuning Job

In [8]:
# Create fine-tuning job
print("Creating fine-tuning job...")
print(f"Configuration:")
print(f"  Model: {FINE_TUNING_CONFIG['model']}")
print(f"  Training file: {FINE_TUNING_CONFIG['training_file']}")
print(f"  Validation file: {FINE_TUNING_CONFIG['validation_file']}")
print(f"  Epochs: {FINE_TUNING_CONFIG['hyperparameters']['n_epochs']}")
print(f"  Batch size: {FINE_TUNING_CONFIG['hyperparameters']['batch_size']}")
print(f"  Seed: {FINE_TUNING_CONFIG['seed']}")
print(f"  Suffix: {FINE_TUNING_CONFIG['suffix']}")

try:
    from openai import BadRequestError
except ImportError:
    BadRequestError = Exception

try:
    fine_tune_job = client.fine_tuning.jobs.create(
        training_file=FINE_TUNING_CONFIG["training_file"],
        validation_file=FINE_TUNING_CONFIG["validation_file"],
        model=FINE_TUNING_CONFIG["model"],
        hyperparameters=FINE_TUNING_CONFIG["hyperparameters"],
        suffix=FINE_TUNING_CONFIG["suffix"],
        seed=FINE_TUNING_CONFIG["seed"]
    )
except BadRequestError as e:
    err_body = getattr(e, "body", None) or getattr(e, "response", None) or {}
    if hasattr(err_body, "json"):
        try:
            err_body = err_body.json()
        except Exception:
            err_body = {}
    elif isinstance(err_body, str):
        import json
        try:
            err_body = json.loads(err_body) if err_body else {}
        except Exception:
            err_body = {}
    err_info = err_body.get("error", {}) if isinstance(err_body, dict) else {}
    err_code = err_info.get("code", "")
    err_msg = err_info.get("message", str(e))
    err_str = str(e).lower()
    if err_code == "exceeded_quota" or ("quota" in err_str and "exceeded" in err_str):
        print("\n❌ Quota exceeded (billing / plan limit)")
        print("   ", err_msg)
        print("\n   What to do:")
        print("   • Add a payment method: https://platform.openai.com/account/billing")
        print("   • Check usage: https://platform.openai.com/usage")
        print("   • Fine-tuning requires a paid account with available balance.")
        raise SystemExit(0) from e
    raise

print(f"\n✅ Fine-tuning job created!")
print(f"   Job ID: {fine_tune_job.id}")
print(f"   Status: {fine_tune_job.status}")
print(f"   Created at: {fine_tune_job.created_at}")

# Save job ID for later reference
JOB_ID = fine_tune_job.id
print(f"\n💾 Save this Job ID for monitoring: {JOB_ID}")

Creating fine-tuning job...
Configuration:
  Model: gpt-4o-mini-2024-07-18
  Training file: file-HubMu4aeHoD528JmrpVo4S
  Validation file: file-G7JGvB35BHgFqeuYeCZsWR
  Epochs: 1
  Batch size: 1
  Seed: 42
  Suffix: techgadgets-support

✅ Fine-tuning job created!
   Job ID: ftjob-tN3nlkG54KiphGn72u6LVmWk
   Status: validating_files
   Created at: 1770713593

💾 Save this Job ID for monitoring: ftjob-tN3nlkG54KiphGn72u6LVmWk


## Step 4: Monitor Training Progress

In [10]:
# Monitor the fine-tuning job
print("Monitoring fine-tuning job progress...")
print("(This may take several minutes to hours depending on dataset size)\n")

max_wait_time = 3600  # Maximum wait time in seconds (1 hour)
check_interval = 30   # Check every 30 seconds
elapsed_time = 0

while elapsed_time < max_wait_time:
    job_status = client.fine_tuning.jobs.retrieve(JOB_ID)
    
    print(f"[{elapsed_time}s] Status: {job_status.status}")
    
    if job_status.status == "succeeded":
        print(f"\n✅ Fine-tuning completed successfully!")
        print(f"   Fine-tuned model ID: {job_status.fine_tuned_model}")
        print(f"   Trained tokens: {job_status.trained_tokens}")
        print(f"   Training loss: {getattr(job_status, 'training_loss', 'N/A')}")
        print(f"   Validation loss: {getattr(job_status, 'validation_loss', 'N/A')}")
        
        FINE_TUNED_MODEL_ID = job_status.fine_tuned_model
        print(f"\n💾 Save this Model ID for evaluation: {FINE_TUNED_MODEL_ID}")
        break
    
    elif job_status.status == "failed":
        print(f"\n❌ Fine-tuning failed!")
        if hasattr(job_status, 'error'):
            print(f"   Error: {job_status.error}")
        break
    
    elif job_status.status in ["validating_files", "queued", "running"]:
        if hasattr(job_status, 'training_file'):
            print(f"   Training file: {job_status.training_file}")
        if hasattr(job_status, 'validation_file'):
            print(f"   Validation file: {job_status.validation_file}")
        if hasattr(job_status, 'trained_tokens'):
            print(f"   Trained tokens so far: {job_status.trained_tokens}")
    
    time.sleep(check_interval)
    elapsed_time += check_interval

if elapsed_time >= max_wait_time:
    print(f"\n⏱️  Maximum wait time reached. Job may still be running.")
    print(f"   Check status manually with: client.fine_tuning.jobs.retrieve('{JOB_ID}')")

Monitoring fine-tuning job progress...
(This may take several minutes to hours depending on dataset size)

[0s] Status: succeeded

✅ Fine-tuning completed successfully!
   Fine-tuned model ID: ft:gpt-4o-mini-2024-07-18:personal:techgadgets-support:D7e2cpyK
   Trained tokens: 99476
   Training loss: N/A
   Validation loss: N/A

💾 Save this Model ID for evaluation: ft:gpt-4o-mini-2024-07-18:personal:techgadgets-support:D7e2cpyK


## Step 5: Retrieve Final Model Information

**Note**: If the job is still running, you can run this cell later to check the status and get the model ID.

In [11]:
# Retrieve final job status
try:
    final_job = client.fine_tuning.jobs.retrieve(JOB_ID)
    
    print("Final Job Status:")
    print(f"  Status: {final_job.status}")
    print(f"  Model: {final_job.model}")
    print(f"  Fine-tuned model: {final_job.fine_tuned_model if hasattr(final_job, 'fine_tuned_model') and final_job.fine_tuned_model else 'Not ready yet'}")
    print(f"  Created at: {final_job.created_at}")
    print(f"  Finished at: {getattr(final_job, 'finished_at', 'Still running')}")
    
    if hasattr(final_job, 'trained_tokens'):
        print(f"  Trained tokens: {final_job.trained_tokens}")
    if hasattr(final_job, 'training_loss'):
        print(f"  Training loss: {final_job.training_loss}")
    if hasattr(final_job, 'validation_loss'):
        print(f"  Validation loss: {final_job.validation_loss}")
    
    if final_job.status == "succeeded":
        print(f"\n✅ Fine-tuned model ready: {final_job.fine_tuned_model}")
        FINE_TUNED_MODEL_ID = final_job.fine_tuned_model
        
except Exception as e:
    print(f"Error retrieving job status: {e}")
    print(f"Make sure JOB_ID is set correctly: {JOB_ID if 'JOB_ID' in locals() else 'Not set'}")

Final Job Status:
  Status: succeeded
  Model: gpt-4o-mini-2024-07-18
  Fine-tuned model: ft:gpt-4o-mini-2024-07-18:personal:techgadgets-support:D7e2cpyK
  Created at: 1770713593
  Finished at: 1770714585
  Trained tokens: 99476

✅ Fine-tuned model ready: ft:gpt-4o-mini-2024-07-18:personal:techgadgets-support:D7e2cpyK
